<a href="https://colab.research.google.com/github/YangxuanWu/Python/blob/master/FinallProject2020/M9_PPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/")

Mounted at /content/drive


In [2]:
!ls

 all_data.csv	    importance_list_all_data.csv
 attacks	    importance_list_for_attack_files.csv
'Colab Notebooks'   results
 CSVs		   'Yangxuan Wu.zip'
 feaure_pics


In [6]:
#import the tool library in whole sections
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import metrics
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os
import pandas as pd
import csv
import time
import warnings
import math
warnings.filterwarnings("ignore")

The following program uses the attack files under the "./attacks/" folder as a dataset. The features used are the 4 features with the highest weight for each file, produced by the feature_selection_for_attack_files file. This file applies the following machine learning algorithm to each file 10 times and prints the results of these operations on the screen.

In [4]:
#retrieve data
csv_files=os.listdir("attacks")# Gets the file name in the attack folder and assigns it to a list (csv_files)。
path=".\\attacks\\"
repetition=10

In [7]:
#Build PPN Model
ml_list={"PPN":Perceptron()}#The machine learning algorithms to be used are defined in a dictionary (ml_list).

In [8]:
# the features to be used for each attack type is defined in a dictionary(features).
# the first 4 of the features created by the file "04_1_feature_selection_for_attack_files.py" are used here.
features={"Bot":["Bwd Packet Length Mean","Flow IAT Max","Flow Duration","Flow IAT Min","Label"],
"DDoS":["Bwd Packet Length Std","Total Backward Packets","Fwd IAT Total","Flow Duration","Label"],
"DoS GoldenEye":["Flow IAT Max","Bwd Packet Length Std","Flow IAT Min","Total Backward Packets","Label"],
"DoS Hulk":["Bwd Packet Length Std","Fwd Packet Length Std","Fwd Packet Length Max","Flow IAT Min","Label"],
"DoS Slowhttptest":["Flow IAT Mean","Fwd Packet Length Min","Bwd Packet Length Mean","Total Length of Bwd Packets","Label"],
"DoS slowloris":["Flow IAT Mean","Total Length of Bwd Packets","Bwd Packet Length Mean","Total Fwd Packets","Label"],
"FTP-Patator":["Fwd Packet Length Max","Fwd Packet Length Std","Fwd Packet Length Mean","Bwd Packet Length Std","Label"],
"Heartbleed":["Total Backward Packets","Fwd Packet Length Max","Flow IAT Min","Bwd Packet Length Max","Label"],
"Infiltration":["Fwd Packet Length Max","Fwd Packet Length Mean","Flow Duration","Total Length of Fwd Packets","Label"],
"PortScan":["Flow Bytes/s","Total Length of Fwd Packets","Fwd IAT Total","Flow Duration","Label"],
"SSH-Patator":["Fwd Packet Length Max","Flow Duration","Flow IAT Max","Total Length of Fwd Packets","Label"],
"Web Attack":["Bwd Packet Length Std","Total Length of Fwd Packets","Flow Bytes/s","Flow IAT Max","Label"]}

In [9]:
#time stamp for all processing time
seconds=time.time()

In [10]:
 #this loop runs on the list containing the filenames. Operations are repeated for all attack files:
#for j in csv_files:
print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % ("File","ML algorithm","accuracy","Precision", "Recall" , "F1-score","Time"))# print output header
    
    #read each attack file:
for j in csv_files: 
    feature_list=list(features[j[0:-4]])
    df=pd.read_csv(path+j, usecols=feature_list)

    #Solve the problem of data hollow value and extreme value 
    df=df.fillna(0)
    for col in feature_list:
      m = df.loc[df[col] != np.inf, col].max()
      df[col].replace(np.inf, m, inplace = True)

    #it changes the normal label to "1" and the attack tag to "0" for use in the machine learning algorithm:
    attack_or_not=[]
    for i in df["Label"]: 
        
        if i =="BENIGN":
            attack_or_not.append(1)
        else:
            attack_or_not.append(0)  

    df["Label"]=attack_or_not

    #this section separates the label and the data into two separate pieces, as Label=y Data=X:
    y = df["Label"]  
    del df["Label"]
    feature_list.remove('Label')
    X = df[feature_list]

    #this loop runs on the list containing the machine learning algorithm names:
    for ii in ml_list: 
        precision=[]
        recall=[]
        f1=[]
        accuracy=[]
        t_time=[]

        # This loop allows cross-validation and machine learning algorithm to be repeated 10 times:
        for i in range(repetition): 
            second=time.time()#time stamp for processing time

            # cross-validation:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = repetition)
            # data (X) and labels (y) are divided into 2 parts to be sent to the machine learning algorithm (80% train,%20 test). 
            # So, in total there are 4 tracks: training data(X_train), training tag (y_train), test data(X_test) and test tag(y_test).

            #machine learing algorithm is applied in this section:
            #choose algorithm from ml_list dictionary 
            clf = ml_list[ii]                                                                        
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
        
            #makes "classification report" and assigns the precision, f-measure, and recall values:   
            f_1=f1_score(y_test, predict, average='macro')
            pr=precision_score(y_test, predict, average='macro')
            rc=recall_score(y_test, predict, average='macro')
            precision.append(float(pr))
            recall.append(float(rc))
            f1.append(float(f_1))
            accuracy.append(clf.score(X_test, y_test))
            t_time.append(float((time.time()-second)))

        #the result of the ten repetitions is printed on the screen:
        print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % (j[0:-4],ii,str(round(np.mean(accuracy),2)),str(round(np.mean(precision),2)),str(round(np.mean(recall),2)),str(round(np.mean(f1),2)),str(round(np.mean(t_time),4))))
        

print("mission accomplished!")
print("Total operation time: = ",time.time()- seconds ,"seconds")


File              ML algorithm       accuracy        Precision       Recall          F1-score        Time           
Bot               PPN                0.71            0.85            0.5             0.42            0.0125         
DDoS              PPN                0.7             0.53            0.5             0.41            0.3384         
DoS GoldenEye     PPN                0.29            0.65            0.5             0.23            0.051          
DoS Hulk          PPN                0.86            0.85            0.79            0.81            1.8556         
DoS Slowhttptest  PPN                0.71            0.73            0.51            0.44            0.0677         
DoS slowloris     PPN                0.48            0.67            0.62            0.48            0.0826         
FTP-Patator       PPN                0.6             0.34            0.43            0.38            0.0843         
Heartbleed        PPN                0.86            0.92       

The following program uses the attack files under the "all_data.csv" as a dataset. The features used are the 18 features with the highest weight for it, produced by the feature_selection_for_attack_files file. This file applies the following machine learning algorithm to the file 10 times and prints the results of these operations on the screen.

In [11]:
#retrieve data:
csv_files=["all_data.csv"]# CSV files names: #The name of the dataset files (csv_files).

repetition=10

In [12]:
#The machine learning algorithms to be used are defined in a dictionary (ml_list).
ml_list={"PPN":Perceptron()}

In [13]:
#The choosing feature for all_data.csv:
features={"all_data":["Bwd Packet Length Max","Bwd Packet Length Mean","Bwd Packet Length Std","Flow Bytes/s",
"Flow Duration","Flow IAT Max","Flow IAT Mean","Flow IAT Min","Flow IAT Std","Fwd IAT Total","Fwd Packet Length Max",
"Fwd Packet Length Mean","Fwd Packet Length Min","Fwd Packet Length Std","Total Backward Packets","Total Fwd Packets",
"Total Length of Bwd Packets","Total Length of Fwd Packets","Label"]}

In [14]:
#time stamp for all processing time:
seconds=time.time()

In [15]:
#Read all_data.cav
feature_list=list(features["all_data"])
df=pd.read_csv("all_data.csv",usecols=feature_list)#read an attack file.

In [16]:
#Solve the problem of data hollow value and extreme value
df=df.fillna(0)
for col in feature_list:
  m = df.loc[df[col] != np.inf, col].max()
  df[col].replace(np.inf, m, inplace = True)

In [17]:
#it changes the normal label to "1" and the attack tag to "0" for use in the machine learning algorithm
attack_or_not=[]
for i in df["Label"]:         
  if i =="BENIGN":
    attack_or_not.append(1)
  else:
    attack_or_not.append(0)           
df["Label"]=attack_or_not

In [18]:
#this section separates the label and the data into two separate pieces, as Label=y Data=X 
y = df["Label"]
del df["Label"]
feature_list.remove('Label')
X = df[feature_list]

In [19]:
print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % ("File","ML algorithm","accuracy","Precision", "Recall" , "F1-score","Time"))# print output header
#this loop runs on the list containing the machine learning algorithm name.
for ii in ml_list: 
  precision=[]
  recall=[]
  f1=[]
  accuracy=[]
  t_time=[]

  # This loop allows cross-validation and machine learning algorithm to be repeated 10 times:
  for i in range(repetition):
    second=time.time()#time stamp for processing time

    # cross-validation:
    # data (X) and labels (y) are divided into 2 parts to be sent to the machine learning algorithm (80% train, %20 test).
    # So, in total there are 4 tracks: training data(X_train), training tag (y_train), test data(X_test) and test tag(y_test).
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = repetition)

    #machine learning algorithm is applied in this section
    #choose algorithm from ml_list dictionary
    clf = ml_list[ii]                                                                          
    clf.fit(X_train, y_train)
    predict =clf.predict(X_test)
        
    #makes "classification report" and assigns the precision, f-measure, and recall values:    
    f_1=f1_score(y_test, predict, average='macro')
    pr=precision_score(y_test, predict, average='macro')
    rc=recall_score(y_test, predict, average='macro')
                      
    precision.append(float(pr))
    recall.append(float(rc))
    f1.append(float(f_1))
    accuracy.append(clf.score(X_test, y_test))
    t_time.append(float((time.time()-second)))

  #the result of the ten repetitions is printed on the screen:     
  print ('%-17s %-17s  %-15s %-15s %-15s %-15s %-15s' % ("all_data",ii,str(round(np.mean(accuracy),2)),str(round(np.mean(precision),2)),str(round(np.mean(recall),2)),str(round(np.mean(f1),2)),str(round(np.mean(t_time),4))))

print("mission accomplished!")
print("Total operation time: = ",time.time() - seconds ,"seconds")

File              ML algorithm       accuracy        Precision       Recall          F1-score        Time           
all_data          PPN                0.74            0.61            0.65            0.62            8.8639         
mission accomplished!
Total operation time: =  111.76448726654053 seconds
